In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import ast
from tqdm import tqdm

In [2]:
preprocessed_df_src = r"C:\Users\Kevin\Desktop\hubmap\obj_detect_bvonly_cv_train_valid.xlsx"
preprocessed_df = pd.read_excel(preprocessed_df_src) # this is from preprocessing_df_objdetect.ipynb
preprocessed_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j,fold
0,0,0033bbc76b6b,blood_vessel,"[[[169, 228], [168, 228], [167, 228], [166, 22...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['73387 16 73897 28 74401 38 74911 43 75420 49...,1,10240,43008,3
1,1,0033bbc76b6b,blood_vessel,"[[[1, 59], [0, 59], [0, 58], [0, 57], [0, 56],...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['30 31 541 32 1053 31 1566 27 2079 25 2594 21...,1,10240,43008,4
2,2,0033bbc76b6b,blood_vessel,"[[[406, 511], [405, 511], [404, 511], [403, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['190976 1 191485 4 191995 6 192506 7 193017 8...,1,10240,43008,4
3,3,00656c6f2690,blood_vessel,"[[[511, 426], [511, 426], [510, 426], [510, 42...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['257947 6 258457 12 258967 16 259478 18 25998...,1,10240,46080,0
4,4,00656c6f2690,blood_vessel,"[[[157, 404], [156, 404], [155, 404], [154, 40...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['58240 8 58750 13 59261 15 59772 16 60284 17 ...,1,10240,46080,3
...,...,...,...,...,...,...,...,...,...,...
3493,3493,fd2437954fd8,blood_vessel,"[[[481, 454], [480, 454], [479, 454], [478, 45...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['238002 14 238510 20 239012 32 239520 37 2400...,1,5120,39424,4
3494,3494,fd2437954fd8,blood_vessel,"[[[416, 511], [415, 511], [414, 511], [413, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['191998 3 192502 11 193012 13 193522 15 19403...,1,5120,39424,4
3495,3495,fd2437954fd8,blood_vessel,"[[[18, 362], [17, 362], [16, 362], [16, 361], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['3365 50 3874 66 4385 70 4896 72 5407 74 5918...,1,5120,39424,4
3496,3496,fe248458ea89,blood_vessel,"[[[131, 501], [130, 501], [129, 501], [128, 50...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['61412 4 61922 8 62432 10 62444 6 62943 21 63...,1,10240,44032,3


### To train our yolov8 model, we need to make a config.yaml, which needs to contain paths for the train and valid dataset images (we don't have a test, that's on kaggle!). Train should be: datasets/train/images, and datasets/train/labels, Valid should be: datasets/val/images, and datasets/val/labels!

In [3]:
yolov8_src = r"C:\Users\Kevin\Desktop\hubmap\yolov8" # change this to wherever!
# make directory first:
train_img_src = os.path.join(yolov8_src,"train","images")
train_label_src = os.path.join(yolov8_src,"train","labels")
val_img_src = os.path.join(yolov8_src,"val","images")
val_label_src = os.path.join(yolov8_src,"val","labels")
if not os.path.exists(train_img_src):
    os.mkdir(train_img_src)
if not os.path.exists(train_label_src):
    os.mkdir(train_label_src)
if not os.path.exists(val_img_src):
    os.mkdir(val_img_src)
if not os.path.exists(val_label_src):
    os.mkdir(val_label_src)

In [4]:
### Here, let's try fold 0 = fold 0 as validation dataset and the rest as training (80-20 split)
val_df = preprocessed_df[preprocessed_df["fold"] == 0]
val_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j,fold
3,3,00656c6f2690,blood_vessel,"[[[511, 426], [511, 426], [510, 426], [510, 42...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['257947 6 258457 12 258967 16 259478 18 25998...,1,10240,46080,0
10,10,00656c6f2690,blood_vessel,"[[[315, 321], [314, 321], [313, 321], [312, 32...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['139054 10 139562 16 140073 18 140582 21 1410...,1,10240,46080,0
12,12,0067d5ad2250,blood_vessel,"[[[318, 511], [317, 511], [316, 511], [315, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['125952 1 126463 2 126973 4 127484 5 127995 6...,2,23552,22528,0
21,21,00da70813521,blood_vessel,"[[[375, 91], [374, 91], [373, 91], [372, 91], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['172618 6 173128 10 173639 12 174150 14 17466...,1,10240,46592,0
29,29,0596bfb19322,blood_vessel,"[[[221, 313], [220, 313], [220, 312], [219, 31...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['103205 9 103709 24 104217 30 104725 35 10523...,1,7680,40448,0
...,...,...,...,...,...,...,...,...,...,...
3470,3470,f86347534ec1,blood_vessel,"[[[102, 427], [101, 427], [100, 427], [100, 42...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['43912 7 44422 23 44933 26 45444 29 45956 30 ...,2,16896,20992,0
3478,3478,f86347534ec1,blood_vessel,"[[[370, 314], [369, 314], [368, 314], [367, 31...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['186679 5 187188 8 187699 9 188210 10 188721 ...,2,16896,20992,0
3486,3486,faba1bf818ae,blood_vessel,"[[[164, 64], [163, 64], [162, 64], [161, 64], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['62978 1 63489 5 64001 9 64513 10 65025 12 65...,1,3072,39424,0
3489,3489,fc6def641612,blood_vessel,"[[[66, 511], [65, 511], [64, 511], [63, 511], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['9727 2 10236 5 10744 9 11252 13 11761 16 122...,1,7680,40960,0


In [5]:
train_df = preprocessed_df[preprocessed_df["fold"] != 0]
train_df

,Unnamed: 0,id,type,coordinates,mask,rle,source_wsi,i,j,fold
0,0,0033bbc76b6b,blood_vessel,"[[[169, 228], [168, 228], [167, 228], [166, 22...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['73387 16 73897 28 74401 38 74911 43 75420 49...,1,10240,43008,3
1,1,0033bbc76b6b,blood_vessel,"[[[1, 59], [0, 59], [0, 58], [0, 57], [0, 56],...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['30 31 541 32 1053 31 1566 27 2079 25 2594 21...,1,10240,43008,4
2,2,0033bbc76b6b,blood_vessel,"[[[406, 511], [405, 511], [404, 511], [403, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['190976 1 191485 4 191995 6 192506 7 193017 8...,1,10240,43008,4
4,4,00656c6f2690,blood_vessel,"[[[157, 404], [156, 404], [155, 404], [154, 40...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['58240 8 58750 13 59261 15 59772 16 60284 17 ...,1,10240,46080,3
5,5,00656c6f2690,blood_vessel,"[[[39, 371], [38, 371], [37, 371], [36, 371], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['1365 14 1874 20 2383 26 2894 28 3405 30 3917...,1,10240,46080,2
...,...,...,...,...,...,...,...,...,...,...
3493,3493,fd2437954fd8,blood_vessel,"[[[481, 454], [480, 454], [479, 454], [478, 45...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['238002 14 238510 20 239012 32 239520 37 2400...,1,5120,39424,4
3494,3494,fd2437954fd8,blood_vessel,"[[[416, 511], [415, 511], [414, 511], [413, 51...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['191998 3 192502 11 193012 13 193522 15 19403...,1,5120,39424,4
3495,3495,fd2437954fd8,blood_vessel,"[[[18, 362], [17, 362], [16, 362], [16, 361], ...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['3365 50 3874 66 4385 70 4896 72 5407 74 5918...,1,5120,39424,4
3496,3496,fe248458ea89,blood_vessel,"[[[131, 501], [130, 501], [129, 501], [128, 50...",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,['61412 4 61922 8 62432 10 62444 6 62943 21 63...,1,10240,44032,3


In [6]:
real_img_src = r"C:\Users\Kevin\Desktop\hubmap\train"
val_img_list = val_df["id"]
val_img_list = [os.path.join(real_img_src,x + ".tif") for x in val_img_list]
train_img_list = train_df["id"]
train_img_list = [os.path.join(real_img_src,x + ".tif") for x in train_img_list]

In [13]:
for idx in tqdm(range(len(val_img_list))):
    val_img = val_img_list[idx]
    shutil.copy(val_img,val_img_src)

100%|██████████| 700/700 [00:01<00:00, 687.22it/s]


In [14]:
for idx in tqdm(range(len(train_img_list))):
    train_img = train_img_list[idx]
    shutil.copy(train_img,train_img_src)

100%|██████████| 2798/2798 [00:01<00:00, 1818.00it/s]


In [154]:
### Now make a function to create .txt file for the bbox annotations for yolov8:
def create_label_txt_file(coordinates, ids, path):
    """
    :df: train_df or val_df from above
    :param ids: each ids from df above
    :param path: save path
    :return: desired yolov8 yaml file
    """
    label_txt = ''
    for coordinate in coordinates:
        label_txt += '0 '
        coor_array = np.array(coordinate[0]).astype(float)
        coor_array /= float(512)
        # transform to str
        coor_list = list(coor_array.reshape(-1).astype(str))
        coor_str = ' '.join(coor_list)
        # add string to label txt
        label_txt += f'{coor_str}\n'

    # Write labels to txt file
    with open(f'{path}/{ids}.txt', 'w') as f:
        f.write(label_txt)

In [156]:
num_images = train_df.shape[0]
coordinates = []
for idx in range(num_images):
    coord = train_df.iloc[idx]["coordinates"]
    coord = ast.literal_eval(coord)
    coordinates.append(coord)
for idx1 in range(train_df.shape[0]):
    create_label_txt_file(coordinates = coordinates, ids = train_df.iloc[idx1]["id"], path = train_label_src)

KeyboardInterrupt: 

In [ ]:
num_images = val_df.shape[0]
coordinates = []
for idx in range(num_images):
    coord = val_df.iloc[idx]["coordinates"]
    coord = ast.literal_eval(coord)
    coordinates.append(coord)
for idx1 in range(val_df.shape[0]):
    create_label_txt_file(coordinates = coordinates, ids = val_df.iloc[idx1]["id"], path = val_label_src)